<h1>Movie Description Text Pre-Processing</h1>

In [1]:
import pandas as pd
import requests
import json
import unidecode
# import goslate
# import sys
# import os
# # project_root = "D:\\Leaning_NLP"
# # sys.path.append(os.path.abspath(os.path.join(os.path.dirname(project_root), 'polyglot')))
# import polyglot
# from polyglot.text import Text
# from transliterate import translit, get_available_language_codes

<h2>Collecting Data</h2>

In [2]:
URL = "https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page="
movies = []
for i in range(100): 
    print(f"\rCollecting movies from page : {i+1}", end = '\r')
    lst = json.loads(requests.get(URL + f"{i+1}").text)['results']
    for movie in lst: 
        movies.append(movie)
print("Collection Completed!!        ", end = "\n")    

Collection Completed!!        100


In [3]:
# movies[0]
GENRE_URL = "https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"
lst = json.loads(requests.get(GENRE_URL).text)['genres']
mp = {}
for item in lst:
    mp[item['id']] = item['name']

In [4]:
lst = []
for movie in movies:
    txt = {}
    txt['original_language'] = movie['original_language']
    txt['original_title'] = movie['original_title']
    # if txt['original_language'] != 'en':
    #     txt['original_title'] = unidecode.unidecode(movie['original_title']).lower()
    txt['overview'] = movie['overview']
    txt['genre'] = []
    for id in movie['genre_ids']:
        txt['genre'].append(mp[id])
    lst.append(txt)
    
df = pd.DataFrame(lst)
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,en,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,en,12 Angry Men,The defense and the prosecution have rested an...,[Drama]
5,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
6,ja,千と千尋の神隠し,"A young girl, Chihiro, becomes trapped in a st...","[Animation, Family, Fantasy]"
7,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[Drama, Action, Crime, Thriller]"
8,ko,기생충,"All unemployed, Ki-taek's family takes peculia...","[Comedy, Thriller, Drama]"
9,en,The Green Mile,A supernatural tale set on death row in a Sout...,"[Fantasy, Drama, Crime]"


In [22]:
# df['original_language'].value_counts()

In [6]:
for r in range(6):
    print(df['original_title'][r])

The Shawshank Redemption
The Godfather
The Godfather Part II
Schindler's List
12 Angry Men
दिलवाले दुल्हनिया ले जायेंगे


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = Options()
chrome_options.add_experimental_option("detach",True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
driver.maximize_window()


from bs4 import BeautifulSoup
import requests

URL = "https://www.themoviedb.org/search?query="

def convert_english(qury):
    try:
        html_text = requests.get(URL + qury).text
        soup = BeautifulSoup(html_text, 'lxml')
        res_box = soup.find('div', class_ = 'results flex')
        res = res_box.find('div', class_ = 'wrapper')
        res = res.find('a')
        # print(res['href'])
        href = res['href']
        forv_url = "https://www.themoviedb.org/"
        href = forv_url + href
        html_text = requests.get(href).text
        soup = BeautifulSoup(html_text, 'lxml')
        res_lang = soup.find('html')['lang']
        if(res_lang != 'en'):
            driver.get(href)
            # button = driver.find_element('xpath', '/html/body/div[1]/header/div[1]/div/div[2]/ul/li[2]')
            # button.click()
            # button = driver.find_element('xpath', "//*[contains(text(), 'English (en-US)')]")
            html_text = driver.page_source
            soup = BeautifulSoup(html_text, 'lxml')
        res = soup.find('section', class_ = 'header poster')
        res = res.find('h2')
        return res.text
    except:
        print("Issue With", qury)
        return qury
        
# print(convert_english('기생충'))

# df['original_title'] = df.apply(lambda x : convert_english(x['original_title'], x['original_language']), axis = 1)
for i in range(df.shape[0]):
    if df['original_language'][i] in ['en', 'fr', 'it', 'es', 'de']:
        continue
    df['original_title'][i] = convert_english(df['original_title'][i])

df.head(10)

driver.quit()

C:\Users\HP\AppData\Local\Temp\ipykernel_15880\91702479.py:56: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['original_title'][i] = convert_english(df['original_title'][i])


Issue With 달콤한 인생
Issue With Spoorloos
Issue With あん
Issue With 葉問2
Issue With Něco z Alenky
Issue With درباره الی‎‎
Issue With 花木兰
Issue With 신세계
Issue With Oslo, 31. august
Issue With 功夫
Issue With 劇場版 NARUTO -ナルト- 疾風伝
Issue With 大鱼海棠
Issue With Mies vailla menneisyyttä
Issue With キュア
Issue With 君たちはどう生きるか
Issue With 鉄コン筋クリート
Issue With 卧虎藏龍
Issue With బాహుబలి 2: ది కన్ క్లూజన్
Issue With 新世紀エヴァンゲリオン劇場版 シト新生
Issue With Vargtimmen
Issue With Serbuan Maut
Issue With ドライブ・マイ・カー
Issue With 醉拳二
Issue With Левиафан
Issue With 人狼 JIN-ROH
Issue With 英雄本色
Issue With MEMORIES
Issue With 버닝
Issue With GHOST IN THE SHELL／攻殻機動隊2.0
Issue With 醉拳
Issue With バイオハザード：デスアイランド
Issue With バブル
Issue With 猛龍過江
Issue With アイアムアヒーロー
Issue With En kongelig affære
Issue With Битва за Севастополь
Issue With Tudo Bem no Natal Que Vem
Issue With 헤어질 결심
Issue With 황해
Issue With ももへの手紙
Issue With おもひでぽろぽろ
Issue With Нелюбовь
Issue With Sala samobójców
Issue With 劇場版 転生したらスライムだった件 紅蓮の絆編
Issue With ドラゴンボール 最強への道
Iss

In [8]:
for i in range(df.shape[0]):
    df.iloc[i, 1] = df.iloc[i, 1].replace('\n', " ")
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,en,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,en,12 Angry Men,The defense and the prosecution have rested an...,[Drama]
5,hi,Dilwale Dulhania Le Jayenge (1995),"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
6,ja,Spirited Away (2001),"A young girl, Chihiro, becomes trapped in a st...","[Animation, Family, Fantasy]"
7,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[Drama, Action, Crime, Thriller]"
8,ko,Parasite (2019),"All unemployed, Ki-taek's family takes peculia...","[Comedy, Thriller, Drama]"
9,en,The Green Mile,A supernatural tale set on death row in a Sout...,"[Fantasy, Drama, Crime]"


<!-- ![alt text]("Screenshot 2024-06-09 203944.png") -->
<h2>PreProcessing Text</h2>
<h4>We will do preprocessing of data on the overview tag</h4>
<h4>Method Used:</h4>
<img src = "Screenshot 2024-06-09 203944.png">

<h3>Lowercasing</h3>

In [12]:
df['overview'] = df['overview'].apply(lambda x : x.lower())
df.sample(10)

,original_language,original_title,overview,genre
455,fr,Z,"amidst a heated political climate, the opposit...","[Thriller, Crime, Drama]"
1644,de,Das Experiment,das experiment is a shocking psycho thriller a...,"[Drama, Thriller]"
603,en,Teen Titans: Trouble in Tokyo,america's coolest heroes teen titans head to j...,"[Animation, Action, Science Fiction, TV Movie]"
1371,en,Men of Honor,against formidable odds -- and an old-school d...,[Drama]
1811,en,Saving Mr. Banks,author p.l. travers looks back on her childhoo...,"[Comedy, Drama, History]"
1259,en,Days of Heaven,"in 1916, a chicago steel worker accidentally k...","[Drama, Romance]"
316,ja,Ugetsu (1953),"in 16th century japan, peasants genjuro and to...","[Fantasy, Drama, Mystery]"
129,es,Voces inocentes,"a young boy, in an effort to have a normal chi...","[Drama, War]"
1313,en,Scott Pilgrim vs. the World,"as bass guitarist for a garage-rock band, scot...","[Action, Comedy, Romance]"
1598,en,Tarzan,tarzan was a small orphan who was raised by an...,"[Family, Adventure, Animation, Drama]"


<h3>Remove HTML tags</h3>

In [18]:
import re
def tags_remover(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

df['overview'] = df['overview'].apply(lambda x : tags_remover(x))
df.sample(10)

,original_language,original_title,overview,genre
808,en,The Gentlemen,american expat mickey pearson has built a high...,"[Action, Crime, Comedy]"
546,ja,Millennium Actress (2002),documentary filmmaker genya tachibana has trac...,"[Animation, Drama, Romance, Fantasy]"
1217,en,E.T. the Extra-Terrestrial,an alien is left behind on earth and saved by ...,"[Science Fiction, Adventure, Family, Fantasy]"
1548,en,Enter the Dragon,a martial artist agrees to spy on a reclusive ...,[Action]
109,ja,Woman in the Dunes (1964),a vacationing entomologist suffers extreme phy...,"[Drama, Thriller]"
1424,fr,En corps,elise thought she had the perfect life: an ide...,"[Comedy, Drama]"
126,en,Hacksaw Ridge,"wwii american army medic desmond t. doss, who ...","[Drama, History, War]"
1744,fr,Frantz,"in the aftermath of wwi, a young german who gr...","[History, Drama, Romance]"
848,en,King Richard,the story of how richard williams served as a ...,"[Drama, History]"
652,en,Misery,after stalking and saving the life of her favo...,"[Drama, Thriller]"


<h3>Remove URLs </h3>

In [21]:
def remove_urls(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

df['overview'] = df['overview'].apply(remove_urls)
df.sample(10)

,original_language,original_title,overview,genre
1762,en,A Boy Called Christmas,an ordinary young boy called nikolas sets out ...,"[Family, Adventure, Fantasy]"
1326,bs,No Man's Land (2020),two soldiers from opposite sites get stuck bet...,"[Action, History, War]"
718,en,Temple Grandin,"a biopic of temple grandin, an autistic americ...","[Drama, History]"
1320,en,Johnny Got His Gun,"a young american soldier, rendered in pseudoco...","[War, Drama]"
695,en,One Life,british stockbroker nicholas winton visits cze...,"[History, Drama, War]"
1003,en,Toy Story 2,"andy heads off to cowboy camp, leaving his toy...","[Animation, Comedy, Family]"
562,en,Harry Potter and the Goblet of Fire,when harry potter's name emerges from the gobl...,"[Adventure, Fantasy]"
99,ja,I Want to Eat Your Pancreas (2018),after his classmate and crush is diagnosed wit...,"[Animation, Drama, Romance]"
1607,en,Injustice,"when lois lane is killed, an unhinged superman...","[Animation, Science Fiction, Fantasy, Action]"
1169,en,Where Eagles Dare,"world war ii is raging, and an american genera...","[Action, Adventure, War]"


<h3>Remove Punctuation</h3>

In [33]:
punctuation_marks = ".;:{}?,-[]'"
def remove_punctuations(text):
    return text.translate(str.maketrans('', '', punctuation_marks))

df['overview'] = df['overview'].apply(remove_punctuations)
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,en,The Godfather,spanning the years 1945 to 1955 a chronicle of...,"[Drama, Crime]"
2,en,The Godfather Part II,in the continuing saga of the corleone crime f...,"[Drama, Crime]"
3,en,Schindler's List,the true story of how businessman oskar schind...,"[Drama, History, War]"
4,en,12 Angry Men,the defense and the prosecution have rested an...,[Drama]
5,hi,Dilwale Dulhania Le Jayenge (1995),raj is a rich carefree happygolucky second gen...,"[Comedy, Drama, Romance]"
6,ja,Spirited Away (2001),a young girl chihiro becomes trapped in a stra...,"[Animation, Family, Fantasy]"
7,en,The Dark Knight,batman raises the stakes in his war on crime w...,"[Drama, Action, Crime, Thriller]"
8,ko,Parasite (2019),all unemployed kitaeks family takes peculiar i...,"[Comedy, Thriller, Drama]"
9,en,The Green Mile,a supernatural tale set on death row in a sout...,"[Fantasy, Drama, Crime]"


<h3>Chat Word Treatment</h3>